In [1]:
import ee
from ee import batch
## Initialize connection to server
ee.Initialize()

In [ ]:
collection = ee.ImageCollection('LANDSAT/LC8_L1T_TOA')
nps_bounds = bound_geometry(ll,ur)
# We can use that object to apply a filter to the world_collection to only include images
# that intersect the spatial bounds we define.   
cbe_collection = collection.filterBounds(nps_bounds)

cc_cr = cbe_collection.filter(ee.Filter.lt('CLOUD_COVER', 1))

tr_cr = cc_cr.filter(ee.Filter.date('2016-01-01', '2017-01-01'))

#plt.imshow(tr_cr, cmap='gray')
#plt.show()

print(collection_length(tr_cr), "images available in monterey_collection.")

In [ ]:
dates_of_images = dates_available(tr_cr)
print("Dates available:",dates_of_images)

# # I also created a function to query which bands are available in a collection.
available_bands_in_monterey = available_bands(tr_cr)
for k, v in available_bands_in_monterey.items():
    print("Band", k, "is available in", v['number_available'], "images. (", v['percent_available'], "%)")

In [ ]:
ll=(76.7, 10.75)
ur=(77.2, 11.25)

In [2]:
%matplotlib inline
%load_ext autoreload 
%autoreload 2
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import scipy.misc
import numpy as np
import fiona
from shapely import geometry
import pprint as pp
import ee
ee.Initialize()

from gee_library import *

import pprint as pp

ll=(76.7, 10.75)
ur=(77.2, 11.25)

In [ ]:
collection = ee.ImageCollection('LANDSAT/LC8_L1T_TOA')
nps_bounds = bound_geometry(ll,ur)

grid_collection = collection.filterBounds(nps_bounds)

tr_cr = grid_collection.filter(ee.Filter.date('2015-01-01', '2017-01-01'))

cc_cr = tr_cr.filter(ee.Filter.lt('CLOUD_COVER', 2))


try:
    tiles,img_affine = img_at_region(geCollection=cc_cr,resolution=30,bands=['B2', 'B3', 'B4', 'B5'],geo_bounds=nps_bounds)
except: 
    print "no suitable image for this region is found during this period, may be try extend the duration of search"

In [ ]:
pixels=100
np_band_array = [scipy.misc.imresize(tiles[b], (pixels, pixels)) for b in ['B4', 'B3', 'B2']]
    
# and stack the images in a matrix
img = np.dstack(np_band_array)
# Display the image
plt.imshow(img); plt.show()



In [ ]:
ndvi=(tiles['B5']-tiles['B4'])/(tiles['B5']+tiles['B4'])

a_ndvi = np.where(ndvi<0.5,0,1)